In [1]:
# Import libraries and modules
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil
print tf.__version__
print np.__version__
print pd.__version__
np.set_printoptions(threshold=np.inf)

1.8.0
1.14.0
0.22.0


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# change these to try this notebook out
BUCKET = 'youtube8m-4-train'
PROJECT = 'qwiklabs-gcp-8d3d0cd07cef9252'
REGION = 'us-central1'

In [3]:
# Import os environment variables
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# Local Development

In [4]:
# Set logging verbosity to INFO for richer output
tf.logging.set_verbosity(tf.logging.INFO)

In [5]:
# The number of video classes
NUM_CLASSES = 4716

In [6]:
arguments = {}
arguments["train_file_pattern"] = "gs://youtube-8m-team/1/video_level/train/train*.tfrecord"
arguments["eval_file_pattern"] = "gs://youtube-8m-team/1/video_level/validate/validate-0.tfrecord"
arguments["output_dir"] = "trained_model"
arguments["batch_size"] = 10
arguments["train_steps"] = 100
arguments["hidden_units"] = [1024, 256, 64]
arguments["top_k"] = 5
arguments["start_delay_secs"] = 60
arguments["throttle_secs"] = 30

In [7]:
# Create an input function to read our training and validation data
# Then provide the results to the Estimator API
def read_dataset_video(file_pattern, mode, batch_size):
  def _input_fn():
    print("\nread_dataset_video: _input_fn: file_pattern = {}".format(file_pattern))
    print("read_dataset_video: _input_fn: mode = {}".format(mode))
    print("read_dataset_video: _input_fn: batch_size = {}".format(batch_size))

    # This function will decode frame examples from the frame level TF Records
    def decode_example(serialized_examples):
      # Create feature map
      feature_map = {
          'video_id': tf.FixedLenFeature(shape = [], dtype = tf.string),
          'labels': tf.VarLenFeature(dtype = tf.int64),
          'mean_rgb': tf.FixedLenFeature(shape = [1024], dtype = tf.float32),
          'mean_audio': tf.FixedLenFeature(shape = [128], dtype = tf.float32)
      }

      # Parse TF Records into our features
      features = tf.parse_single_example(serialized = serialized_examples, features = feature_map)
      print("\nread_dataset_video: _input_fn: decode_example: features = {}".format(features)) # shape = video_id = (), mean_rgb = (1024,), mean_audio = (128,), labels = SparseTensor object

      # Extract and format labels
      sparse_labels = features.pop("labels") # SparseTensor object
      print("read_dataset_video: _input_fn: decode_example: sparse_labels = {}\n".format(sparse_labels))
      labels = tf.cast(x = tf.sparse_to_dense(sparse_indices = sparse_labels.values, output_shape = (NUM_CLASSES,), sparse_values = 1, validate_indices = False), dtype = tf.float32)
      print("read_dataset_video: _input_fn: decode_example: labels = {}\n".format(labels)) # shape = (NUM_CLASSES,)

      return features, labels

    # Create list of files from file pattern
    file_list = tf.gfile.Glob(filename = file_pattern)
    #print("read_dataset_video: _input_fn: file_list = {}".format(file_list))

    # Create dataset from file list
    dataset = tf.data.TFRecordDataset(filenames = file_list)
    print("read_dataset_video: _input_fn: dataset.TFRecordDataset = {}".format(dataset))

    # Decode TF Record dataset examples
    dataset = dataset.map(map_func = lambda x: decode_example(serialized_examples = x))
    print("read_dataset_video: _input_fn: dataset.map = {}".format(dataset))

    # Determine amount of times to repeat file and if we should shuffle based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # read files forever
      
      # Shuffle the dataset within a buffer
      dataset = dataset.shuffle(buffer_size = batch_size * 10, seed = None)
      print("read_dataset_video: _input_fn: dataset.shuffle = {}".format(dataset))
    else:
      num_epochs = 1 # read files only once

    # Repeat files num_epoch times
    dataset = dataset.repeat(count = num_epochs)
    print("read_dataset_video: _input_fn: dataset.repeat = {}".format(dataset))

    # Group the data into batches
    dataset = dataset.batch(batch_size = batch_size)
    print("read_dataset_video: _input_fn: dataset.batch = {}".format(dataset))

    # Create a iterator and then pull the next batch of features and labels from the example queue
    batch_features, batch_labels = dataset.make_one_shot_iterator().get_next()
    print("read_dataset_video: _input_fn: batch_features = {}".format(batch_features))
    print("read_dataset_video: _input_fn: batch_labels = {}\n".format(batch_labels))

    return batch_features, batch_labels
  return _input_fn

In [18]:
def try_input_function():
  with tf.Session() as sess:
    fn = read_dataset_video(file_pattern = "gs://youtube-8m-team/1/video_level/train/train*.tfrecord", mode = tf.estimator.ModeKeys.TRAIN, batch_size = 5)
    batch_features, batch_labels = fn()
    features, labels = sess.run([batch_features, batch_labels])
    
    print("\ntry_input_function: features = {}".format(features))
    print("try_input_function: labels = {}\n".format(labels))

In [19]:
try_input_function()


read_dataset_video: _input_fn: file_pattern = gs://youtube-8m-team/1/video_level/train/train*.tfrecord
read_dataset_video: _input_fn: mode = train
read_dataset_video: _input_fn: batch_size = 5
read_dataset_video: _input_fn: dataset.TFRecordDataset = <TFRecordDataset shapes: (), types: tf.string>

read_dataset_video: _input_fn: decode_example: features = {'labels': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f2b7c7b0fd0>, 'video_id': <tf.Tensor 'ParseSingleExample/ParseSingleExample:5' shape=() dtype=string>, 'mean_audio': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=(128,) dtype=float32>, 'mean_rgb': <tf.Tensor 'ParseSingleExample/ParseSingleExample:4' shape=(1024,) dtype=float32>}
read_dataset_video: _input_fn: decode_example: sparse_labels = SparseTensor(indices=Tensor("ParseSingleExample/ParseSingleExample:0", shape=(?, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseSingleExample:1", shape=(?,), dtype=int64), dense_shape=Tensor("Pars

In [8]:
# Create our model function to be used in our custom estimator
def video_level_model(features, labels, mode, params):
  print("\nvideo_level_model: features = {}".format(features))
  print("video_level_model: labels = {}".format(labels))
  print("video_level_model: mode = {}".format(mode))

  # 0. Configure network
  # Get dynamic batch size
  current_batch_size = tf.shape(features['mean_rgb'])[0]
  print("video_level_model: current_batch_size = {}".format(current_batch_size))

  # Stack all of the features into a 3-D tensor
  combined_features = tf.concat(values = [features['mean_rgb'], features['mean_audio']], axis = 1) # shape = (current_batch_size, 1024 + 128)
  print("video_level_model: combined_features = {}".format(combined_features))

  # 1. Create the DNN structure now
  # Create the input layer to our frame DNN
  network = combined_features # shape = (current_batch_size, 1024 + 128)
  print("video_level_model: network = combined_features = {}".format(network))

  # Add hidden layers with the given number of units/neurons per layer
  for units in params['hidden_units']:
    network = tf.layers.dense(inputs = network, units = units, activation = tf.nn.relu) # shape = (current_batch_size, units)
    print("video_level_model: network = {}, units = {}".format(network, units))

  # Connect the final hidden layer to a dense layer with no activation to get the logits
  logits = tf.layers.dense(inputs = network, units = NUM_CLASSES, activation = None) # shape = (current_batch_size, NUM_CLASSES)
  print("video_level_model: logits = {}".format(logits))

  # Select the top k logits in descending order
  top_k_logits = tf.nn.top_k(input = logits, k = params['top_k'], sorted = True) # shape = (current_batch_size, top_k)
  print("video_level_model: top_k_logits = {}".format(top_k_logits))

  # Since this is a multi-class, multi-label problem we will apply a sigmoid, not a softmax, to each logit to get its own probability
  probabilities = tf.sigmoid(logits) # shape = (current_batch_size, NUM_CLASSES)
  print("video_level_model: probabilities = {}".format(probabilities))

  # Select the top k probabilities in descending order
  top_k_probabilities = tf.sigmoid(top_k_logits.values) # shape = (current_batch_size, top_k)
  print("video_level_model: top_k_probabilities = {}".format(top_k_probabilities))

  # Select the top k classes in descending order of likelihood
  top_k_classes = top_k_logits.indices # shape = (current_batch_size, top_k)
  print("video_level_model: top_k_classes = {}".format(top_k_classes))

  # The 0/1 predictions based on a threshold, in this case the threshold is if the probability it greater than random chance
  predictions = tf.where(
    condition = probabilities > 1.0 / NUM_CLASSES, # shape = (current_batch_size, NUM_CLASSES)
    x = tf.ones_like(tensor = probabilities), 
    y = tf.zeros_like(tensor = probabilities))
  print("video_level_model: predictions = {}".format(predictions))

  top_k_predictions = tf.where(
    condition = top_k_probabilities > 1.0 / NUM_CLASSES, # shape = (current_batch_size, top_k)
    x = tf.ones_like(tensor = top_k_probabilities), 
    y = tf.zeros_like(tensor = top_k_probabilities))
  print("video_level_model: top_k_predictions = {}\n".format(top_k_predictions))

  # 2. Loss function, training/eval ops 
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    # Since this is a multi-class, multi-label problem, we will use sigmoid activation and cross entropy loss
    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels = labels, logits = logits)

    train_op = tf.contrib.layers.optimize_loss(
      loss = loss,
      global_step = tf.train.get_global_step(),
      learning_rate = 0.01,
      optimizer = "Adam")
    eval_metric_ops = {
      "accuracy": tf.metrics.mean_per_class_accuracy(labels = labels, predictions = predictions, num_classes = NUM_CLASSES)
    }
  else:
    loss = None
    train_op = None
    eval_metric_ops = None

  # 3. Create predictions
  predictions_dict = {"logits": top_k_logits.values, 
                      "probabilities": top_k_probabilities, 
                      "predictions": top_k_predictions,
                      "classes": top_k_classes}

  # 4. Create export outputs
  export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = predictions_dict)}

  # 5. Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
      mode = mode,
      predictions = predictions_dict,
      loss = loss,
      train_op = train_op,
      eval_metric_ops = eval_metric_ops,
      export_outputs = export_outputs)

In [62]:
# Create our serving input function to accept the data at serving and send it in the right format to our custom estimator
def serving_input_fn():
  # This function fixes the shape and type of our input strings
  def fix_shape_and_type_for_serving(placeholder):
    # String split each string in the batch and output the values from the resulting SparseTensors
    split_string = tf.map_fn(
      fn = lambda x: tf.string_split(source = [placeholder[x]], delimiter=',').values, 
      elems = tf.range(start = 0, limit = tf.shape(input = placeholder)[0]), 
      dtype = tf.string) # shape = (batch_size, input_sequence_length)
    print("serving_input_fn: fix_shape_and_type_for_serving: split_string = {}".format(split_string))

    # Convert each string in the split tensor to float
    feature_tensor = tf.string_to_number(string_tensor = split_string, out_type = tf.float32) # shape = (batch_size, input_sequence_length)
    print("serving_input_fn: fix_shape_and_type_for_serving: feature_tensor = {}".format(feature_tensor))
    return feature_tensor
  
  # This function fixes dynamic shape ambiguity of last dimension so that we will be able to use it in our DNN (since tf.layers.dense require the last dimension to be known)
  def get_shape_and_set_modified_shape_2D(tensor, additional_dimension_sizes):
    # Get static shape for tensor and convert it to list
    shape = tensor.get_shape().as_list()
    # Set outer shape to additional_dimension_sizes[0] since we know that this is the correct size
    shape[1] = additional_dimension_sizes[0]
    # Set the shape of tensor to our modified shape
    tensor.set_shape(shape = shape) # shape = (batch_size, additional_dimension_sizes[0])
    print("serving_input_fn: get_shape_and_set_modified_shape_2D: tensor = {}, additional_dimension_sizes = {}".format(tensor, additional_dimension_sizes))
    return tensor
  
  # Create placeholders to accept the data sent to the model at serving time
  feature_placeholders = { # all features come in as a batch of strings, shape = (batch_size,), this was so because of passing the arrays to online ml-engine prediction
    'video_id': tf.placeholder(dtype = tf.string, shape = [None]),
    'mean_rgb': tf.placeholder(dtype = tf.string, shape = [None]),
    'mean_audio': tf.placeholder(dtype = tf.string, shape = [None])
  }
  print("\nserving_input_fn: feature_placeholders = {}".format(feature_placeholders))

  # Create feature tensors
  features = {
    "video_id": feature_placeholders["video_id"],
    "mean_rgb": fix_shape_and_type_for_serving(placeholder = feature_placeholders["mean_rgb"]),
    "mean_audio": fix_shape_and_type_for_serving(placeholder = feature_placeholders["mean_audio"])
  }
  print("serving_input_fn: features = {}".format(features))

  # Fix dynamic shape ambiguity of feature tensors for our DNN
  features["mean_rgb"] = get_shape_and_set_modified_shape_2D(tensor = features["mean_rgb"], additional_dimension_sizes = [1024])
  features["mean_audio"] = get_shape_and_set_modified_shape_2D(tensor = features["mean_audio"], additional_dimension_sizes = [128])
  print("serving_input_fn: features = {}\n".format(features))
  
  return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [63]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(args):
  # Create our custome estimator using our model function
  estimator = tf.estimator.Estimator(
    model_fn = video_level_model, 
    model_dir = args['output_dir'],
    params = {'hidden_units': args['hidden_units'], 'top_k': args['top_k']})
  # Create train spec to read in our training data
  train_spec = tf.estimator.TrainSpec(
    input_fn = read_dataset_video(
      file_pattern = args['train_file_pattern'], 
      mode = tf.estimator.ModeKeys.TRAIN, 
      batch_size = args['batch_size']),
    max_steps = args['train_steps'])
  # Create exporter to save out the complete model to disk
  exporter = tf.estimator.LatestExporter(name = 'exporter', serving_input_receiver_fn = serving_input_fn)
  # Create eval spec to read in our validation data and export our model
  eval_spec = tf.estimator.EvalSpec(
    input_fn = read_dataset_video(
      file_pattern = args['eval_file_pattern'], 
      mode = tf.estimator.ModeKeys.EVAL, 
      batch_size = args['batch_size']),
    steps = None,
    exporters = exporter,
    start_delay_secs = args['start_delay_secs'],
    throttle_secs = args['throttle_secs'])
  # Create train and evaluate loop to train and evaluate our estimator
  tf.estimator.train_and_evaluate(estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)

In [64]:
# Run the training job
shutil.rmtree(arguments['output_dir'], ignore_errors = True) # start fresh each time
train_and_evaluate(args = arguments)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f238cd690>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained_model', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 30 secs (eval_spec.throttle_secs) or training is finished.

read_dataset_video: _input_fn: file_pattern = gs://youtube-8m-team/1/video_level/train/train*.tfrecord
read_dataset_video: _input_

# Training

### Locally

In [65]:
%bash
OUTDIR=trained_model
rm -rf $OUTDIR
export PYTHONPATH=$PYTHONPATH:$PWD/trainer
python -m trainer.task \
  --train_file_pattern="gs://youtube-8m-team/1/video_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/video_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --batch_size=10 \
  --train_steps=100 \
  --hidden_units="1024 512 256" \
  --top_k=5 \
  --job-dir=./tmp


read_dataset_video: _input_fn: file_pattern = gs://youtube-8m-team/1/video_level/train/train*.tfrecord
read_dataset_video: _input_fn: mode = train
read_dataset_video: _input_fn: batch_size = 10
read_dataset_video: _input_fn: dataset.TFRecordDataset = <TFRecordDataset shapes: (), types: tf.string>

read_dataset_video: _input_fn: decode_example: features = {'labels': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f4307390610>, 'video_id': <tf.Tensor 'ParseSingleExample/ParseSingleExample:5' shape=() dtype=string>, 'mean_audio': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=(128,) dtype=float32>, 'mean_rgb': <tf.Tensor 'ParseSingleExample/ParseSingleExample:4' shape=(1024,) dtype=float32>}
read_dataset_video: _input_fn: decode_example: sparse_labels = SparseTensor(indices=Tensor("ParseSingleExample/ParseSingleExample:0", shape=(?, 1), dtype=int64), values=Tensor("ParseSingleExample/ParseSingleExample:1", shape=(?,), dtype=int64), dense_shape=Tensor("Par

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4307444f90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained_model/', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running tr

### GCloud

In [20]:
%bash
OUTDIR=gs://$BUCKET/youtube_8m_video_level_datasets/trained_model
JOBNAME=job_youtube_8m_video_level_datasets$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.5 \
  -- \
  --train_file_pattern="gs://youtube-8m-team/1/video_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/video_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --batch_size=50 \
  --train_steps=10000 \
  --hidden_units="1024 512 256" \
  --top_k=5 \
  --job-dir=$OUTDIR

gs://youtube8m-4-train/youtube_8m_video_level_datasets/trained_model us-central1 job_youtube_8m_video_level_datasets180525_002728
jobId: job_youtube_8m_video_level_datasets180525_002728
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [job_youtube_8m_video_level_datasets180525_002728] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe job_youtube_8m_video_level_datasets180525_002728

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs job_youtube_8m_video_level_datasets180525_002728


### Hyperparameter tuning

In [66]:
%writefile hyperparam.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: accuracy
    goal: MAXIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32", "256 128 16", "64 64 64 8"]

Writing hyperparam.yaml


In [67]:
%bash
OUTDIR=gs://$BUCKET/youtube_8m_video_level_datasets/hyperparam
JOBNAME=job_youtube_8m_video_level_datasets$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --config=hyperparam.yaml \
  --runtime-version=1.5 \
  -- \
  --train_file_pattern="gs://youtube-8m-team/1/video_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/video_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --train_steps=10000 \
  --top_k=5 \
  --job-dir=$OUTDIR

gs://youtube8m-4-train/youtube_8m_video_level_datasets/hyperparam us-central1 job_youtube_8m_video_level_datasets180525_061024
jobId: job_youtube_8m_video_level_datasets180525_061024
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [job_youtube_8m_video_level_datasets180525_061024] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe job_youtube_8m_video_level_datasets180525_061024

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs job_youtube_8m_video_level_datasets180525_061024


# Deploy

In [41]:
%bash
MODEL_NAME="youtube_8m_video_level_datasets"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://$BUCKET/youtube_8m_video_level_datasets/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create $MODEL_NAME --regions $REGION
gcloud ml-engine versions create $MODEL_VERSION --model $MODEL_NAME --origin $MODEL_LOCATION --runtime-version 1.5

Deleting and deploying youtube_8m_video_level_datasets v1 from gs://youtube8m-4-train/youtube_8m_video_level_datasets/trained_model/export/exporter/1527208584/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-8d3d0cd07cef9252/models/youtube_8m_video_level_datasets].
Creating version (this might take a few minutes)......
..............................................................................................................done.


# Prediction

### Prep

In [ ]:
# Let's call our input function to decode our data to put into BigQuery for testing predictions
video_list = []
with tf.Session() as sess:
  fn = read_dataset_video(
    file_pattern = "gs://youtube-8m-team/1/video_level/validate/validate-0.tfrecord", 
    mode = tf.estimator.ModeKeys.EVAL, 
    batch_size = 1)
  batch_features, batch_labels = fn()
  for key,value in batch_features.items():
    batch_features[key] = tf.squeeze(batch_features[key])
  fixed_batch_features = batch_features
  fixed_batch_labels = tf.squeeze(batch_labels)

  while True:
    features, labels = sess.run([fixed_batch_features, fixed_batch_labels])
    features["labels"] = labels
    video_list.append(features)

In [22]:
# This is the number of videos from the video level file we just processed
len(video_list)

331

In [25]:
# Convert the nd-arrays to lists and cast to strings (video_id is already a single string)
for items in video_list:
  items["labels"] = str(items["labels"].tolist())
  items["mean_rgb"] = str(items["mean_rgb"].tolist())
  items["mean_audio"] = str(items["mean_audio"].tolist())

In [26]:
# Create a dataframe from the list
video_df = pd.DataFrame(video_list)
video_df = video_df[["video_id", "mean_rgb", "mean_audio", "labels"]]
video_df.head()

,video_id,mean_rgb,mean_audio,labels
0,-0AjrnQ0N3A,"[0.31275367736816406, -0.0275861918926239, -0....","[-1.3102529048919678, -0.16714175045490265, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,-0MpgkddrY4,"[0.3996831178665161, -0.7902982831001282, 1.85...","[1.4524954557418823, 1.5982519388198853, 1.783...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,-02RMEBtLDo,"[-0.5323705077171326, -0.18418750166893005, 1....","[-0.3671940267086029, 0.43636152148246765, 1.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,-00LLAtj0JE,"[-0.12762920558452606, -0.9175933599472046, -0...","[0.5460372567176819, 1.4538171291351318, -1.59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,-07wapPiIAg,"[-1.5168684720993042, 1.3144418001174927, 0.65...","[0.9732653498649597, -0.26707080006599426, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [27]:
# Export dataframe to BigQuery
import datalab.bigquery as bq
bigquery_dataset_name = 'ryan_youtube'
bigquery_table_name = 'tbl_video_level'

# Define BigQuery dataset and table\n",
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists\n",
table_schema = bq.Schema.from_dataframe(video_df)
table.create(schema = table_schema, overwrite = True)

video_df.to_gbq(destination_table = bigquery_dataset_name + '.' + bigquery_table_name, project_id = "qwiklabs-gcp-8d3d0cd07cef9252", if_exists = "replace")




Load is 100% Complete




In [28]:
# Create SQL query
query="""
SELECT
  video_id,
  mean_rgb,
  mean_audio
FROM
  `qwiklabs-gcp-8d3d0cd07cef9252.ryan_youtube.tbl_video_level`
LIMIT
  3
"""

In [29]:
# Export BigQuery results to dataframe
import google.datalab.bigquery as bq2
df_predict = bq2.Query(query).execute().result().to_dataframe()
df_predict.head()

,video_id,mean_rgb,mean_audio
0,-0FDy3F9Gqo,"[0.5610039830207825, 0.6452203989028931, 0.365...","[-1.1589237451553345, 0.8263356685638428, 0.21..."
1,-0CK7kXtP-U,"[0.18520089983940125, -0.03229318931698799, -0...","[0.4032512307167053, -0.6461716294288635, 0.08..."
2,-0QTVHnvT90,"[0.5585809350013733, 0.09270916134119034, -0.2...","[-0.4419490396976471, -0.3150809705257416, 0.7..."


### Local prediction from local model

In [48]:
# Format dataframe to new line delimited json strings and write out to json file
with open('video_level.json', 'w') as outfile:
  for idx, row in df_predict.iterrows():
    json_string = "{\"video_id\": \"" + row["video_id"] + "\", \"mean_rgb\": \"" + row["mean_rgb"].replace(" ","").replace("[","").replace("]","") + "\", \"mean_audio\": \"" + row["mean_audio"].replace(" ","").replace("[","").replace("]","") + "\"}"
    outfile.write("%s\n" % json_string)

In [57]:
%bash
model_dir=$(ls $PWD/trained_model/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=$PWD/trained_model/export/exporter/$model_dir \
    --json-instances=./video_level.json

CLASSES           LOGITS                                                                                                   PREDICTIONS                PROBABILITIES
[0, 1, 4, 2, 13]  [-1.3381117582321167, -2.023491144180298, -2.0803487300872803, -2.235565185546875, -2.2626023292541504]  [1.0, 1.0, 1.0, 1.0, 1.0]  [0.20782074332237244, 0.11675848811864853, 0.11102154105901718, 0.09660188108682632, 0.09426794201135635]
[0, 1, 4, 13, 2]  [-1.62063729763031, -2.3859150409698486, -2.425746440887451, -2.4261155128479004, -2.68080472946167]     [1.0, 1.0, 1.0, 1.0, 1.0]  [0.16511699557304382, 0.08425307273864746, 0.08123035728931427, 0.08120281249284744, 0.06411556899547577]
[0, 1, 4, 13, 2]  [-1.658339500427246, -2.566516160964966, -2.623415231704712, -2.7371816635131836, -2.8348546028137207]   [1.0, 1.0, 1.0, 1.0, 1.0]  [0.1599850207567215, 0.07132472097873688, 0.0676465705037117, 0.06081467494368553, 0.05546950548887253]


  from ._conv import register_converters as _register_converters
2018-05-25 04:56:36.734122: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



### GCloud ML-Engine prediction from deployed model

In [36]:
# Format dataframe to instances list to get sent to ML-Engine
instances = [{"video_id": row["video_id"], "mean_rgb": row["mean_rgb"].replace(" ","").replace("[","").replace("]",""), "mean_audio": row["mean_audio"].replace(" ","").replace("[","").replace("]","")} for idx, row in df_predict.iterrows()]

In [37]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials)

request_data = {"instances": instances}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'youtube_8m_video_level_datasets', 'v1')
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {u'predictions': [{u'probabilities': [0.17853404581546783, 0.1317114382982254, 0.10703973472118378, 0.08211781829595566, 0.07856766134500504], u'logits': [-1.5263111591339111, -1.8859106302261353, -2.1213419437408447, -2.4139139652252197, -2.4619691371917725], u'classes': [0, 1, 3, 2, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}, {u'probabilities': [0.17853404581546783, 0.1317114382982254, 0.10703973472118378, 0.08211781829595566, 0.07856766134500504], u'logits': [-1.5263111591339111, -1.8859106302261353, -2.1213419437408447, -2.4139139652252197, -2.4619691371917725], u'classes': [0, 1, 3, 2, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}, {u'probabilities': [0.17853404581546783, 0.1317114382982254, 0.10703973472118378, 0.08211781829595566, 0.07856766134500504], u'logits': [-1.5263111591339111, -1.8859106302261353, -2.1213419437408447, -2.4139139652252197, -2.4619691371917725], u'classes': [0, 1, 3, 2, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}]}
